# Personalised Movie Recommender

Here is presented a user-centered system to find movies similar to ones that they like, but may not have heard of

### Import required moduels and investigate downloaded datasets.

In [1]:
import pandas as pd
import os
import glob, os
import numpy as np

path = "C:/Users/charris/Desktop/datasets/movie_data/ml-25m/"

os.chdir(path)
for file in glob.glob("*.csv"):
    print(file)

genome-scores.csv
genome-tags.csv
links.csv
movies.csv
ratings.csv
tags.csv


The two files we need are  movies.csv and ratings.csv

## Create DataFrame objects
This will likely cause memory errors due to size of the ratings files. we can limit this file and still generate good predctions

In [2]:
movies = pd.read_csv(path + "movies.csv")
#ratings = pd.read_csv(path + "ratings.csv")

In [3]:
ratings = pd.read_csv(path + "ratings.csv",skiprows = list(range(75000, 25000096)))

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Merge two dataframes and clean

In [7]:
movie_ratings = pd.merge(ratings, movies, on='movieId', how='left')

We now have a single dataframe containing all the information we need for our program

In [8]:
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


For me, a film's release year shouldshuold be seperate from the title. THis will make it easier for the user to search for films.

We copy over last 6 characters from the title strings into a new column and the use regex to remove all charaters inside parentheses from the original series 

In [9]:
movie_ratings['year'] = movie_ratings['title'].str[-6:]

In [10]:
movie_ratings['title'] = movie_ratings['title'].str.replace(r" ?\([^)]+\)", "")

In [11]:
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,296,5.0,1147880044,Pulp Fiction,Comedy|Crime|Drama|Thriller,(1994)
1,1,306,3.5,1147868817,Three Colors: Red,Drama,(1994)
2,1,307,5.0,1147868828,Three Colors: Blue,Drama,(1993)
3,1,665,5.0,1147878820,Underground,Comedy|Drama|War,(1995)
4,1,899,3.5,1147868510,Singin' in the Rain,Comedy|Musical|Romance,(1952)


<!-- ## Compress size

To make the program useable we will need to cut some of the size. For this we can place a limit in the number of users -->

In [12]:
movie_list = movie_ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movie_list.head()

rating          
                  size      mean
title                           
 Days of Summer   37.0  3.972973
$5 a Day           1.0  4.000000
'71                1.0  2.500000
'Round Midnight    1.0  4.000000
'Salem's Lot       2.0  3.000000

## Create correlation between movies

We need to pivot the table to get the film titles on the column and user ID on the axis

In [19]:
movie_rating_pivot = movie_ratings.drop(columns = ['movieId', 'timestamp', 'genres'])

In [20]:
movie_rating_pivot.tail()

,userId,rating,title,year
74994,572,3.0,"Replacements, The",(2000)
74995,572,4.0,"Cell, The",(2000)
74996,572,4.5,"Naked Gun: From the Files of Police Squad!, The",(1988)
74997,572,3.5,Almost Famous,(2000)
74998,572,3.0,Fantastic Voyage,(1966)


In [21]:
movie_rating_pivot = movie_rating_pivot.pivot_table(columns=['title'], index = ['userId'], values = 'rating')

In [22]:
movie_rating_pivot.head()

title,Days of Summer,$5 a Day,'71,'Round Midnight,'Salem's Lot,'Til There Was You,"'burbs, The",*batteries not included,...And Justice for All,10,...,loudQUIETloud: A Film About the Pixies,xXx,xXx: State of the Union,¡Three Amigos!,Кентервильское привидение,Когда зажигаются ёлки,Обезьянки и грабители,Он вам не Димон,Приключения Домовёнка,Принцесса и Людоед
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We set a condition that each film has to be rated by a user at least 25 times. This will allow better predictions.

In [51]:
CorrMatrix = movie_rating_pivot.corr(method='pearson', min_periods=25)
CorrMatrix

title,Days of Summer,$5 a Day,'71,'Round Midnight,'Salem's Lot,'Til There Was You,"'burbs, The",*batteries not included,...And Justice for All,10,...,loudQUIETloud: A Film About the Pixies,xXx,xXx: State of the Union,¡Three Amigos!,Кентервильское привидение,Когда зажигаются ёлки,Обезьянки и грабители,Он вам не Димон,Приключения Домовёнка,Принцесса и Людоед
title,,,,,,,,,,,,,,,,,,,,,
Days of Summer,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$5 a Day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Когда зажигаются ёлки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Обезьянки и грабители,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Он вам не Димон,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN values indicate that the correlated movies have not been rated enough times to be included in the matrix.

## Recommender
The user enters in a film title to return names of similar movies. Te number of which is given by variable: films

In [52]:
films = 20
filmname = "Matrix, The"

In [53]:
print('Here are some ideas for films you may not have seen but are similar to {}....'.format(filmname))
cm = CorrMatrix.loc[:,filmname].sort_values(ascending=False)[:films]
cm2 = cm.drop(labels=filmname)
cm3 = cm2.to_frame().dropna()
print()
print(cm3)

Here are some ideas for films you may not have seen but are similar to Matrix, The....

                                 Matrix, The
title                                       
Thank You for Smoking               0.736578
Predator                            0.655570
Mummy, The                          0.643804
Crow, The                           0.634340
Sleepy Hollow                       0.633579
Training Day                        0.623431
Star Trek: First Contact            0.588251
Aliens                              0.575150
Big Fish                            0.561764
Unbreakable                         0.560685
The Butterfly Effect                0.559178
Beetlejuice                         0.543826
Mr. & Mrs. Smith                    0.539573
300                                 0.538803
Sin City                            0.528499
Prestige, The                       0.508294
Monty Python and the Holy Grail     0.506417
Citizen Kane                        0.498446
The Devil's 